## Libraries imports

Firts step we need to import all libraries we going to use, like:

- <strong>requests</strong> used to make our http requests at the page  where we want to scrape data.
- <strong>fromstring</strong> used to parse our html page into a string.
- <strong>cycle</strong> used to create a cycle of proxies for our requests.
- <strong>BeautifulSoup</strong> used to parse, filter by tag and scrap specific data from the web page.
- <strong>pandas</strong> used to interact with the tables and lists colected from the web page and export them.
- <strong>time & random</strong> 
- <strong>clear_output</strong> 
- <strong>warnings</strong> 

In [19]:
# Libraries imports
from lxml.html import fromstring
from itertools import cycle
from bs4 import BeautifulSoup
from IPython.display import clear_output
import requests as req
import pandas as pd
import time, random
import warnings as wrns

## Defining Folder and Headers

In [2]:
# Folder to save scraped files
folder = "G:/Google Drive/Colab Notebooks/investHub/data/raw/"

# Request Headers
headers = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
              'Accept-Encoding': 'deflate, br',
              'Accept-Language': 'pt-BR,pt;q=0.9,en;q=0.8',
              'Cache-Control': 'max-age=0',
              'Connection': 'keep-alive',
              #'Content-Length': '254',
              #'Content-Type': 'application/x-www-form-urlencoded',
              #'Cookie': 'ASP.NET_SessionId=5yx0ptz53yt1mibxavmqyb1v; uid=9112324E72B36CDCAF433C566D19FCC6; origem=https://www.oceans14.com.br/acoes/; _ga=GA1.3.1992174690.1623332100; _gid=GA1.3.1928600702.1623332100; Sessao=3933DA519B26445B10C8521140884D75; tipoUsuario=UsuÃ¡rio; contadorViewsAcoes=0',
              #'Host': 'www.oceans14.com.br',
              #'Origin': 'https://www.oceans14.com.br',
              #'Referer': 'https://www.oceans14.com.br/acoes/',
              #'Sec-Fetch-Mode': 'navigate',
              #'Sec-Fetch-Site': 'same-origin',
              #'Sec-Fetch-User': '?1',
              #'Upgrade-Insecure-Requests': '1',
              'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36'
              }

In [13]:
# Function to Scrap proxies
def get_proxies(qtd_min):
    url = 'https://www.sslproxies.org/'
    proxies = set()
    while True:
        response = requests.get(url)
        parser = fromstring(response.text)
        for i in parser.xpath('//tbody/tr')[:100]:
            if i.xpath('.//td[7][contains(text(),"yes")]'):
                proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
                proxies.add(proxy)
        if len(proxies) < qtd_min:
            for sleeping in range(600,0,-1):#Time to free proxies refresh
                time.sleep(1)
                clear_output()
                print(str(len(proxies)/qtd_min*100) + '%')
                print('New Request in ' + str(sleeping) + '...')
            continue
        else:
            break
    clear_output()
    print('Done...' + str(len(proxies)) + 'proxies!')
    return proxies

In [53]:
# Scraping url for all B3 Stocks data
url = 'https://www.oceans14.com.br/acoes'
page = req.get(url, headers=headers, timeout=15)
soup = BeautifulSoup(page.content, 'html.parser')
url_list = []
for url in soup.find_all('a'):
    if 'balanco-dividendos' in url.get('href'):
        url_list.append('https://www.oceans14.com.br' + url.get('href'))

url_list = list(set(url_list))
url_list_loop = url_list
print(len(url_list))
print(url_list)

256
['https://www.oceans14.com.br/acoes/lavvi/lavv3/balanco-dividendos', 'https://www.oceans14.com.br/acoes/hospital-mater-dei/matd3/balanco-dividendos', 'https://www.oceans14.com.br/acoes/aes-tiete/aesb3/balanco-dividendos', 'https://www.oceans14.com.br/acoes/anima-educacao/anim3/balanco-dividendos', 'https://www.oceans14.com.br/acoes/usiminas/usim/balanco-dividendos', 'https://www.oceans14.com.br/acoes/coelce/coce/balanco-dividendos', 'https://www.oceans14.com.br/acoes/raia-drogasil/radl3/balanco-dividendos', 'https://www.oceans14.com.br/acoes/bb-seguridade/bbse3/balanco-dividendos', 'https://www.oceans14.com.br/acoes/localiza/rent3/balanco-dividendos', 'https://www.oceans14.com.br/acoes/banrisul/brsr/balanco-dividendos', 'https://www.oceans14.com.br/acoes/vale/vale3/balanco-dividendos', 'https://www.oceans14.com.br/acoes/klabin/klbn/balanco-dividendos', 'https://www.oceans14.com.br/acoes/petrobras/petr/balanco-dividendos', 'https://www.oceans14.com.br/acoes/hypera/hype3/balanco-divi

In [54]:
# Save url_list to a csv(opcional)
url_list_df = pd.DataFrame(url_list)
url_list_df.to_csv(folder + 'url_list.txt', index=False, sep=';')

In [69]:
proxies = get_proxies(2*len(url_list))
proxy_pool = cycle(proxies)

In [67]:
# Save proxies to a csv(opcional)
proxies_df = pd.DataFrame(proxies)
proxies_df.to_csv(folder + 'proxies.txt', index=False, sep=';')

In [52]:
# Exec in case of Exception Break(opcional)
proxies_df = pd.read_csv('G:/Google Drive/Colab Notebooks/investHub/data/raw/proxies.txt', sep=';')
proxies_df = pd.Series(proxies_df.iloc[:,0])
proxies = proxies_df.values.tolist()
proxy_pool = cycle(proxies)

In [28]:
# Exec in case of Exception Break(opcional)
df = pd.read_csv('G:/Google Drive/Colab Notebooks/investHub/data/raw/url_list.csv', sep=';')
df = pd.Series(df.iloc[:,0])
url_list = df.values.tolist()
url_list_loop = url_list

In [72]:
wrns.filterwarnings('ignore')
# Get data for each stocks
for url in url_list_loop:
    ticker_list = []
    while True:
        proxy = next(proxy_pool)
        #time.sleep(random.randint(2, 10))
        try:
            ticker_req = req.get(url, headers=headers, timeout=5, verify=False, proxies={"http": proxy, "https": proxy})
            ticker_page = ticker_req.content
            soup = BeautifulSoup(ticker_page, 'html.parser')
            ticker_data = pd.read_html(ticker_page, decimal=",", thousands=".", header=0, match= 'P/VP')
            url_list.remove(url)
            print(len(url_list))
            # Get stock's ticker
            for url in soup.find_all('h4'):
                ticker_list.append(url.get_text('h4'))
            
            # Historical Fundamental Analysis indicators
            for i in range(len(ticker_list)):
                td_hist = ticker_data[i]
                filename = ticker_list[i] + '.csv'
                path = folder + filename
                td_hist.to_csv(path, index=False, sep=';')
                print(filename)
              
        except Exception as ex:
            #print(ex)
            #print('Connection Failed...')
            continue
        else:
            break

0
GSHP3.csv
